This script evaluates trained machine learning models (Lasso, HGBR) and a climatology baseline on a seasonal test dataset. It loads the test data, prepares features and targets, loads the trained models from disk, fits a climatology model, and evaluates all models. The evaluation results are saved for further analysis.

In [2]:
import sys
import os
sys.path.append('/home/vgarcia/notebooks')

from models_functions import *

In [3]:
# Input file
experiment_name = "Optimize_ML_all"
trained_model_path = f"/home/vgarcia/experiments/ML_seasonal_new/{experiment_name}/"
preprocessing_path = "/data/dl20-data/climate_operational/Victor_data/preprocessed_datasets_ML_monthly_new"
models = ["Lasso", "HGBR"]

In [4]:
# Load test dataset
df = pd.read_csv(f"{preprocessing_path}/test/test.csv", index_col=0).reset_index()
target_cols = ["rx90p_anom", "pr_anom"]
y_test = df[target_cols]
X_test = df.drop(columns=target_cols + ["year"])

In [7]:
# load models
models_dict = {}
for model in models:
    models_dict[model] = joblib.load(f"{trained_model_path}{model}_model.pkl")

# create climatology model
clim_model = ClimatologyModel(time_group="annual")
clim_model.fit(X_test, None)
models_dict["Climatology"] = clim_model

evaluate_models(X_test, y_test, models_dict, store_testing=True, out_path = trained_model_path, experiment_name=experiment_name)

---Testing (MSE, MSSS, MAE, R2)---
Lasso: [0.1743875  0.16152009], [-0.03049585 -0.01001914], [0.30267573 0.2960628 ], [0.07524427 0.11326149]
HGBR: [0.17445136 0.16219026], [-0.03087323 -0.01420983], [0.30277219 0.29698027], [0.07490562 0.10958231]
Climatology: [0.16922678 0.15991785], [0. 0.], [0.29700949 0.29300301], [0.10261092 0.12205772]
